## Automate read excel

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel(r'D:\My Documents\Desktop\python_office_11_OCT_2023\src\data\sales_data\2021\August.xlsx')
df

#### read multiple file excel

In [ ]:
import os

xlsx_file_lists = []

path = r'D:\My Documents\Desktop\python_office_11_OCT_2023\src\data\sales_data'

for root,dirs,files in os.walk(path):
    #print(root)
    #print(dirs)
    #print(files)
    for name in files:
        #print(root)
        #print(name)
        file_path = os.path.join(root,name)
       # print(file_path)
        xlsx_file_lists.append(file_path)
print(xlsx_file_lists)

In [ ]:
df = pd.read_excel(xlsx_file_lists[0])
df

In [ ]:
msg = r'D:\My Documents\Desktop\python_office_11_OCT_2023\src\data\sales_data\2021\April.xlsx'

print(msg.split("\\"))
print(msg.split("\\")[-2])

In [ ]:
for f in xlsx_file_lists:
    if f.split("\\")[-2] == "2021":
        print(f)

In [ ]:
for f in xlsx_file_lists:
    if f.split("\\")[-1] == "April.xlsx":
        print(f)

In [ ]:
for f in xlsx_file_lists:
    if f.split("\\")[-1].split(".")[0] == "April":
        print(f)

In [ ]:
for f in xlsx_file_lists:
    if f.split("\\")[-2] == "2022":
        if f.split("\\")[-1].split(".")[0] == "April":
            print(f)

In [ ]:
import datetime 
today = datetime.date.today()
year = today.year-1
#print(year)

for f in xlsx_file_lists:
    if f.split("\\")[-2] == str(year):
        if f.split("\\")[-1].split(".")[0] == "April":
            print(f)
            df = pd.read_excel(f)
df

#### Example sales report summary

In [ ]:
import datetime 
today = datetime.date.today()
year = today.year-1

xlxs_file_lists = []

for root,dirs,files in os.walk(path):
      for name in files:
        file_path = os.path.join(root,name)
        if file_path.split("\\")[-2] == str(year): #change filter
            xlxs_file_lists.append(file_path)
xlxs_file_lists

In [ ]:
df_lists = []

for f in xlxs_file_lists:
    df = pd.read_excel(f)
    df_lists.append(df)
df_lists

In [ ]:
df_summary = pd.concat(df_lists)
df_summary

In [ ]:
pivot = pd.pivot_table(df_summary,index="transaction_date",columns="store",values="amount",aggfunc="sum")
pivot

In [ ]:
summary_monthly = pivot.resample("M").sum()
summary_monthly

In [ ]:
import matplotlib
fig = summary_monthly.plot(kind="bar",figsize=(20,12),fontsize=26,title="monthly sale summary").get_figure()

In [56]:
import xlwings as xw

import datetime
now = datetime.datetime.now()
date_file_name = f'{str(now.date())}_{str(now.time()).split(".")[0].replace(":","_")}'


template = xw.Book(r"D:\My Documents\Desktop\python_office_11_OCT_2023\src\data\sale_template.xlsx")

app = xw.apps.active
sheet = template.sheets["summary"]
sheet["A1"].value = summary_monthly

pivot_page = template.sheets["pivot"]
pivot_page["A1"].value = pivot

#add picture
sheet_report = template.sheets["report"]
sheet_report["A1"].value = "Summary by month"
sheet_report['A1'].font.size = 24
sheet_report["A1"].api.Font.Bold = True
plot= sheet_report.pictures.add(fig,top=sheet["A3"].top,left=sheet["A3"].left)
plot.width = plot.width*0.8
plot.height = plot.height*0.8

template.save(f"export\summary_sale_report_{date_file_name}.xlsx")
template.close()
app.kill()

<>:26: SyntaxWarning: invalid escape sequence '\s'
<>:26: SyntaxWarning: invalid escape sequence '\s'
C:\Users\lbj6639\AppData\Local\Temp\ipykernel_15256\1926154670.py:26: SyntaxWarning: invalid escape sequence '\s'
  template.save(f"export\summary_sale_report_{date_file_name}.xlsx")


#### script node-red
conda activate python_office && python "D:\My Documents\Desktop\python_office\src\sale_report_script.py"

#### install pm2
- npm install -g pm2 
- pm2 status
- pm2 start C:\Users\<User>\AppData\Roaming\npm\node_modules\node-red\red.js
- pm2 start C:\Users\lbj6639\AppData\Roaming\npm\node_modules\node-red\red.js
- npm install pm2-windows-startup -g
- pm2-startup install
- pm2 save

#### send email

In [2]:
#!pip install email

In [3]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText

from datetime import date

today = date.today()

send_from = 'suraphop.b@minebea.co.th'
send_to = ['suraphop.b@minebea.co.th','devops.mic@gmail.com']
subject = f"sale summary report {today}"

text = f'''<html>
        <head><b>Dear sir</b></head>
        <body>
            <div>
              I would like to report summary sale result with a attach file as below
            </div>
            <p>Suraphop Bunsawat<br>MIC division<br>Tel: 2236</p><br>
            <div>
            <i>a attactfile</i>
            </div>
        </body>
    </html>'''

files = f"export\summary_sale_report_{date_file_name}.xlsx"

username = 'suraphop.b@minebea.co.th'
password = ""


msg = MIMEMultipart()
msg['Subject'] = subject 
msg['From'] = send_from
msg['To'] = ', '.join(send_to)

msg.attach(MIMEText(text,'html'))

part = MIMEBase('application', "octet-stream")
with open(files, 'rb') as f:
    file = f.read()
part.set_payload(file)
encoders.encode_base64(part)    
part.add_header('Content-Disposition', f'attachment; filename="sale_summary.xlsx"')
msg.attach(part)

server = smtplib.SMTP("smtp.gmail.com", 587)
server.ehlo()
server.starttls()
server.login(username, password)
server.sendmail(send_from,send_to, msg.as_string())